In [3]:
import pandas as pd
import numpy as np

In [4]:
import os

exp_id = '10-04'
config = '/Users/zhuangweikang/Documents/GitHub/DMSConfig/dms_simulator/%s/drl/ddpg.csv' % exp_id
results = pd.read_csv(config, index_col=0)
avg = np.mean(results, 0).tolist()
config_meta = pd.read_csv('../meta/config_meta.csv')
max_rlt = results.max().tolist()
min_rlt = results.min().tolist()

max_rlt = np.round((config_meta['max'] - config_meta['min']) * max_rlt + config_meta['min'])
min_rlt = np.round((config_meta['max'] - config_meta['min']) * min_rlt + config_meta['min'])
mean_rlt = np.round((config_meta['max'] - config_meta['min']) * avg + config_meta['min'])

print('default:', config_meta['default'].tolist())
print('config_meta_min:', config_meta['min'].tolist())
print('config_meta_max:', config_meta['max'].tolist())
print('reco', mean_rlt.tolist())

default: [3, 8, 500, 102400, 102400, 104857600, 33554432, 0, 16384, 0]
config_meta_min: [1, 1, 50, 10240, 10240, 10485760, 2097152, 0, 4096, 0]
config_meta_max: [20, 24, 5000, 204800, 204800, 314572800, 100663296, 1, 262144, 100]
reco [16.0, 24.0, 5000.0, 185006.0, 67783.0, 12365788.0, 2217413.0, 1.0, 262143.0, 100.0]


In [5]:
from pyDOE import *

meta_info = pd.read_csv('../meta/config_meta.csv')
meta_info['min'] = min_rlt
meta_info['max'] = max_rlt

# generate samples using Latin Hypercube Sampling
def sample_configs(budget):
    global meta_info
    # get the number factors
    num_factors = meta_info.shape[0]
    sampler = lhs(num_factors, samples=budget, criterion="maximin")
    sampler = pd.DataFrame(data=sampler)
    sampler.columns = meta_info['name']
    for col_name in meta_info['name']:
        param_meta = meta_info.loc[meta_info['name'] == col_name]
        if param_meta["data_type"].to_list()[0] == "int":
            sampler[col_name] = sampler[col_name].apply(
                lambda x: int(round(x * (param_meta["max"] - param_meta["min"]) + param_meta["min"])))
        elif param_meta["data_type"].to_list()[0] == "double":
            sampler[col_name] = sampler[col_name].apply(
                lambda x: '%.2f' % float(x * (param_meta["max"] - param_meta["min"]) + param_meta["min"]))
        elif param_meta['data_type'].to_list()[0] in ['string', 'boolean']:
            options = param_meta['options'].to_list()[0].split('/')
            sampler[col_name] = sampler[col_name].apply(
                lambda x: options[int(round(x * (param_meta["max"] - param_meta["min"]) + param_meta["min"]))])
    sampler.to_csv(path_or_buf="schedule.csv", index=False)

sample_configs(100)